In [1]:
# Step 1 — Import Libraries and Load Data
import pandas as pd

# Load your dataset (update the path if needed)
file_path = r'C:\Users\Nexgen\Desktop\Pakistan_Electricity_Data\data\optimization.xlsx'

df = pd.read_excel(file_path)

# Show first few rows and basic info
print("Dataset Preview:\n")
print(df.head())

print("\nDataset Info:\n")
print(df.info())

print("\nMissing Values:\n")
print(df.isnull().sum())


Dataset Preview:

   Year  Total Generation  Residential Consumption  Industrial Consumption  \
0  2000             66562                  25000.0                 15000.0   
1  2001             68117                  25900.0                 15990.0   
2  2002             71653                  27867.0                 16700.0   
3  2003             91595                  29890.0                 17600.0   
4  2004             93607                  30000.0                 18700.0   

   Agricultural Consumption  
0                    6500.0  
1                    6600.0  
2                    6700.0  
3                    6867.0  
4                    6910.0  

Dataset Info:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25 entries, 0 to 24
Data columns (total 5 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Year                      25 non-null     int64  
 1   Total Generation          25 non-null     int

In [2]:
!pip install pulp


  Using cached pulp-3.3.0-py3-none-any.whl (16.4 MB)


You should consider upgrading via the 'C:\Users\Nexgen\Desktop\Pakistan_Electricity_Data\venv\Scripts\python.exe -m pip install --upgrade pip' command.


In [3]:
import pandas as pd
from pulp import LpMaximize, LpProblem, LpVariable, lpSum, LpStatus, value


In [7]:
def optimize_allocation(row, weights):
    # Extract data for the given year
    total_generation = row['Total Generation']

    # Create LP model
    model = LpProblem("Electricity_Allocation", LpMaximize)

    # Decision variables
    R = LpVariable("Residential", lowBound=0)
    I = LpVariable("Industrial", lowBound=0)
    A = LpVariable("Agricultural", lowBound=0)

    # Objective: maximize weighted sum (sector priorities)
    model += weights['R']*R + weights['I']*I + weights['A']*A

    # Constraints: total allocation cannot exceed generation
    model += R + I + A <= total_generation

    # Optional: ensure minimum supply for each sector (to prevent 0 allocation)
    model += R >= 0.1 * total_generation * weights['R']
    model += I >= 0.1 * total_generation * weights['I']
    model += A >= 0.1 * total_generation * weights['A']

    # Solve
    model.solve()

    return {
        'Optimized_Residential': value(R),
        'Optimized_Industrial': value(I),
        'Optimized_Agricultural': value(A),
        'Total_Allocated': value(R + I + A),
        'Status': LpStatus[model.status]
    }


In [8]:
weights = {'R': 0.5, 'I': 0.3, 'A': 0.2}


In [11]:
optimized_results = df.apply(lambda row: pd.Series(optimize_allocation(row, weights)), axis=1)
final_df = pd.concat([df, optimized_results], axis=1)


In [12]:
print(final_df[['Year', 
                'Total Generation', 
                'Optimized_Residential', 
                'Optimized_Industrial', 
                'Optimized_Agricultural', 
                'Total_Allocated', 
                'Status']])


    Year  Total Generation  Optimized_Residential  Optimized_Industrial  \
0   2000             66562               63233.90               1996.86   
1   2001             68117               64711.15               2043.51   
2   2002             71653               68070.35               2149.59   
3   2003             91595               87015.25               2747.85   
4   2004             93607               88926.65               2808.21   
5   2005             88379               83960.05               2651.37   
6   2006             96478               91654.10               2894.34   
7   2007             99895               94900.25               2996.85   
8   2008             98151               93243.45               2944.53   
9   2009             95166               90407.70               2854.98   
10  2010            100369               95350.55               3011.07   
11  2011            101773               96684.35               3053.19   
12  2012             9949

In [13]:
print(final_df[['Year', 
                'Total Generation', 
                'Optimized_Residential', 
                'Optimized_Industrial', 
                'Optimized_Agricultural', 
                'Total_Allocated', 
                'Status']])


    Year  Total Generation  Optimized_Residential  Optimized_Industrial  \
0   2000             66562               63233.90               1996.86   
1   2001             68117               64711.15               2043.51   
2   2002             71653               68070.35               2149.59   
3   2003             91595               87015.25               2747.85   
4   2004             93607               88926.65               2808.21   
5   2005             88379               83960.05               2651.37   
6   2006             96478               91654.10               2894.34   
7   2007             99895               94900.25               2996.85   
8   2008             98151               93243.45               2944.53   
9   2009             95166               90407.70               2854.98   
10  2010            100369               95350.55               3011.07   
11  2011            101773               96684.35               3053.19   
12  2012             9949